In [1]:
import chainLLM
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

In [2]:
model_choice = "ChatGPT-4o"
data = chainLLM.pdf_data("policy-booklet-0923.pdf")
db = chainLLM.create_chunks(data)

/opt/homebrew/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/opt/homebrew/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [31]:
def similarity_score(sentence1,sentence2):
    embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    embedding1 = embeddings_model.embed_query(sentence1)
    embedding2 = embeddings_model.embed_query(sentence2)
    print(embedding1, embedding2)
    embedding1 = np.array(embedding1).reshape(1, -1)
    embedding2 = np.array(embedding2).reshape(1, -1)
    similarity_score = cosine_similarity(embedding1, embedding2)[0][0]
    similarity_score = round(similarity_score, 4)
    return similarity_score


In [36]:
df = pd.read_csv('dataset.csv')
questions = df['Query'].tolist()
answers = df['Response'].tolist()
df_evaluation = {
    'Query': [],
    'Original Response': [],
    'LLM Response': []
}

In [34]:
index = 1
query = str(questions[index])
original_response = str(answers[index]).replace('\'', '')
llm_response = str(chainLLM.get_output(query, db, model_choice)).replace('\'', '')
# similarity_score = similarity_score(original_response,llm_response)

TypeError: 'numpy.float64' object is not callable

In [37]:
for i in range(len(questions)): 
    query = str(questions[i])
    original_response = str(answers[i]).replace('\'', '')
    llm_response = str(chainLLM.get_output(query, db, model_choice)).replace('\'', '')
    # similarity_score = similarity_score(original_response,llm_response)
    df_evaluation['Query'].append(str(query))
    df_evaluation['Original Response'].append(str(original_response))
    df_evaluation['LLM Response'].append(str(llm_response))
    print(f"Query: {query}")

Query: What information does the booklet provide?
Query: What are the benefits included in the policy?
Query: What are some of the exclusions in the policy?
Query: How do I make a claim?
Query: When is the policy renewal notice sent?
Query: Who can I contact for assistance with my policy?
Query: What types of coverage does the policy offer?
Query: Are pre-existing conditions covered by the policy?
Query: How soon should I notify the insurer after an incident?
Query: What should be reviewed during policy renewal?
Query: Is the customer service available 24/7?
Query: What should be included when making a claim?
Query: How does personal accident coverage work in the policy?
Query: What is the critical illness coverage about?
Query: Are self-inflicted injuries covered by the policy?
Query: What are the general exclusions in this policy?
Query: What is the procedure for renewing the policy?
Query: How do I contact customer service?
Query: When should I review my policy terms?
Query: What do

In [38]:
df_evaluation = pd.DataFrame(df_evaluation)
df_evaluation.to_csv('evaluation_metrics_llm_org.csv', index=False)